In [1]:
pip install qsharp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 KB 350.5 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
import qsharp
from Hubbard import EstimateEigenstateEnergy
import matplotlib.pyplot as plt

AttributeError: partially initialized module 'qsharp' has no attribute 'noise_model' (most likely due to a circular import)

In [15]:
show = False

n = 100
bins = 100
xDim = 2
yDim = 2
t = 1.0
U = 0.0
stepSize = 0.1
trotterOrder = 1
bitPrecision = 7

paramStr = str( xDim ) + "x" + str( yDim ) + ", t=" + str (t) + ", U=" + str(U) + ", step =" + str ( stepSize ) + ", order =" + str( trotterOrder ) + ", bitPrec =" + str ( bitPrecision ) + ", n=" + str(n)
x = []
for i in range (n):
    energy = EstimateEigenstateEnergy . simulate ( xDim =xDim , yDim =yDim , t=t, U=U, stepSize = stepSize , trotterOrder = trotterOrder , bitPrecision = bitPrecision )
    x. append ( energy )
    print (i, "/", n, " Energy :", energy )

print (" Energy error from phase estimation :", 2**( - bitPrecision )/ stepSize )
plt . hist (x, bins = bins )
plt . title ( paramStr )

if show == True :
    plt . show ()
else :
    plt . savefig ( paramStr + ". png")

NameError: name 'EstimateEigenstateEnergy' is not defined

In [13]:
namespace Hubbard {
    open Microsoft . Quantum . Intrinsic ;
    open Microsoft . Quantum . Arrays ;
    open Microsoft . Quantum . Characterization ;
    open Microsoft . Quantum . Oracles ;
    open Microsoft . Quantum . Canon ;

    function Void () : Unit {
     }

    // Estimates the eigenstate energy using the 2D Hubbard - Hamiltonian in the half - full case
    operation EstimateEigenstateEnergy (( xDim : Int , yDim : Int), t : Double ,
                                        U : Double , stepSize : Double ,
                                        trotterOrder : Int ,
                                        bitPrecision : Int ) : Double {
        let nSites = xDim*yDim ;

    // Gets the hubbard terms
    let hubbardTerms = GetHubbardTerm (nSites , xDim , t, U, _, _, _);
    // Combines hubbard term application into one operation using the Trotter - Suzuki integrator
    let trotterIntegrator = DecomposedIntoTimeStepsCA ((5* nSites , hubbardTerms ), trotterOrder );
    // Converts trotterIntegrator unitary operation to DiscreteOracle type
    let oracle = OracleToDiscrete ( trotterIntegrator ( stepSize , _));

    using ( qubits = Qubit [2* nSites ]) {
    // Each site is represented by a pair of qubits , one for spin -up
    // and one for spin - down . The qubits can be imagined as being
    // arranged in a so called snake -shape , where the first half of
    // the qubits represents the spin -up modes from site 1 to n and the
    // second half represents the spin - down modes from site n to 1.

    // Initialises the anti - ferromagnetic , half - fullstate
    for ((i, qubit ) in Enumerated ( qubits )) {
        if (i%2 == 0) {X( qubit );}
    }

    // Estimates the phase of the operator which equals the energy times the stepsize
    let energy = RobustPhaseEstimation ( bitPrecision , oracle , qubits )/ stepSize ;
    ResetAll ( qubits );
    return energy ;
        }
    }

    operation GetHubbardTerm ( nSites : Int , xDim : Int , t : Double , U : Double , i : Int ,
                              stepSize : Double , qubits : Qubit []) : Unit is Adj + Ctl {
     // Onsite terms
    if (i < nSites ) {
        GetHubbardOnsiteTerm (nSites , i, U* stepSize , qubits );
    }
    // Array - adjacent hopping terms
    elif (i < 3* nSites - 1 && i != 2* nSites - 1) {
    GetHubbardArrayAdjacentHoppingTerm (nSites , xDim , i - nSites , t* stepSize , qubits );
     }
    // Non -array - adjacent hopping terms
     elif (i >= 3* nSites && (i - 3* nSites + 1) % xDim != 0 && (i < 5* nSites - xDim &&
                                                                 (i < 4* nSites - xDim || i >= 4* nSites ))) {
         GetHubbardNonArrayAdjacentHoppingTerm (nSites , xDim , i - 3* nSites , t* stepSize , qubits );
        }
    }
    

    operation GetHubbardOnsiteTerm ( nSites : Int , i : Int , c : Double , qubits : Qubit []) : Unit is Adj + Ctl {
        Exp ([ PauliI ], 0.25*c, [ qubits [i]]);
        Exp ([ PauliZ ], -0.25*c, [ qubits [i]]);
        Exp ([ PauliZ ], -0.25*c, [ qubits [2* nSites - 1 - i]]);
        Exp ([ PauliZ , PauliZ ], 0.25*c, [ qubits [i], qubits [2* nSites - 1 - i]]);
    }

    operation GetHubbardArrayAdjacentHoppingTerm ( nSites : Int , xDim : Int , i : Int , c : Double , qubits : Qubit []) : Unit is Adj + Ctl {
        let target = qubits [i .. i + 1];
        Exp ([ PauliX , PauliX ], -0.5*c, target );
        Exp ([ PauliY , PauliY ], -0.5*c, target );
    }

    operation GetHubbardNonArrayAdjacentHoppingTerm ( nSites : Int , xDim : Int , i : Int , c : Double , qubits : Qubit []) : Unit is Adj + Ctl {
        let target = qubits [i .. i + 2* xDim - 1 - 2*( i % xDim )];
        Exp ( GetPauliString ( Length ( target ), PauliX ), -0.5*c, target );
        Exp ( GetPauliString ( Length ( target ), PauliY ), -0.5*c, target );
    }

    function GetPauliString ( length : Int , direction : Pauli ) : Pauli [] {
     mutable pauli = ConstantArray (length , PauliZ );
     set pauli w/= 0 <- direction ;
     set pauli w/= length - 1 <- direction ;
     return pauli ;
    }
}

SyntaxError: '{' was never closed (1211622027.py, line 1)